In [ ]:
import tensorflow as tf
import random

In [ ]:
from dgl.data import citation_graph as citegrh
import numpy as np
## load the dataset of citeseer or core
#data = citegrh.load_citeseer()
#data = citegrh.load_cora()
data = citegrh.load_pubmed()

In [ ]:
def random_walks(links,walk_length = 100, iterations=10):
    nodes_map = dict()
    for a,b in links:
        if a not in nodes_map:
            nodes_map[a] = set()
        nodes_map[a].add(b)

    for a in nodes_map:
        nodes_map[a] = list(nodes_map[a])

    paths = []
    for num_walks in range(iterations):
        for a in nodes_map:
            path = []
            now = a
            path.append(now)
            index = random.randint(0,len(nodes_map[now])-1)
            path.append(nodes_map[now][index])
            last = a
            now = nodes_map[now][index]

            for step in range(walk_length-2):       
                if now not in nodes_map:
                    break
                lst = nodes_map[now]
                last = now
                now = random.choice(lst)                                   
                path.append(now)
            paths.append(path)
    return paths

In [ ]:
voc_set = set()
links = []
for a,b in data.graph.edges:
    links.append([a,b])
    voc_set.add(a)
    voc_set.add(b)
num_nodes = len(voc_set)

In [ ]:
sents = random_walks(links)

In [ ]:
window_size = 10
train_xs_data,train_ys_data = [], []
for sent in sents:
    for i in range(len(sent)):
        for j in range(len(sent)):
            if i!=j and (i-j<=window_size and i-j>=-window_size):
                a,b = sent[i],sent[j]
                train_xs_data.append(a)
                train_ys_data.append(b)
train_xs_data = np.array(train_xs_data)
train_ys_data = np.array(train_ys_data)

In [ ]:
#Computational Graph Definition
tf.reset_default_graph()#remove this if not ipython notebook
batch_size = 512
embedding_size = 2 # Dimension of the embedding vector.
num_sampled = 64 # Number of negative examples to sample.


node_embeddings_in = tf.Variable(tf.random_uniform([num_nodes, embedding_size], -1.0, 1.0))
node_embeddings_out = tf.Variable(tf.random_uniform([num_nodes, embedding_size], -1.0,1.0))

#Fixedones
biases=tf.Variable(tf.zeros([num_nodes]))

train_xs =  tf.placeholder(tf.int32, shape=[None],name="xs")
train_ys =  tf.placeholder(tf.int32, shape=[None,1],name="ys")

xs_emb = tf.nn.embedding_lookup(node_embeddings_in, train_xs)



loss_node2vec = tf.reduce_mean(tf.nn.sampled_softmax_loss(node_embeddings_out,
                                                          biases,train_ys,xs_emb, num_sampled, num_nodes))
update_loss = tf.train.AdamOptimizer().minimize(loss_node2vec)

init = tf.initialize_all_variables()


In [ ]:
loss_rec = []
batch_num = len(train_xs_data) // batch_size
with tf.Session() as sess:
    sess.run(init)
    init_lr =0.05
    for epoch in range(1):
        for i in range(batch_num):            
            data_xs =train_xs_data[i*batch_size:(i+1)*batch_size]
            data_ys = train_ys_data[i*batch_size:(i+1)*batch_size].reshape([-1,1])
            feed_dict={
                       train_xs:data_xs,
                       train_ys:data_ys,
                      }        
            _,loss_value=sess.run([update_loss,loss_node2vec], feed_dict)
                 
    Embeddings = sess.run(node_embeddings_in)
                

In [ ]:
np.save('pubmed_structure_2d.npy',Embeddings)